In [0]:
import pandas as pd
from tqdm.notebook import tqdm
import json
import numpy as np

In [0]:
pos_indexer = {POS.POS_list: index for index,POS in pd.read_csv('pos_list.csv').iterrows()}
json.dump(pos_indexer,open('pos_indexer.json','w'))

In [0]:
from tqdm.notebook import tqdm
import torch

class translator :
    def __init__(self, words, min_count, empty=False) :
      if not empty :
        npwords = np.array(words)
        count = [(word,(npwords==word).sum()) for word in tqdm(np.unique(words))]
        count = sorted(count, key=lambda x: x[1], reverse=True)
        d = -1
        while True :
          d += 1
          if count[d][1] < min_count :
            break
        self.dim = d+1
        self.mapping = {count[i][0] : i for i in range(d)}

    def one_hot(self, i) :
      a = torch.zeros(self.dim)
      a[i] = 1
      return a

    def __getitem__(self, i) :
        if type(i)==type('') :
            return self.one_hot(self.mapping.get(i, self.dim-1))
        elif type(i)==type(1) :
            return self.one_hot(i)
        elif type(i)==type(list()) :
            return torch.cat(list(map(self.__getitem__, i)))
        else :
            return self.__getitem__(i.tolist())

    def get_item(self, i) :
        if type(i)==type('') :
            return self.mapping.get(i, self.dim-1)
        if type(i)==type(1) :
            return i
        elif type(i)==type(list()) :
            return torch.tensor(list(map(self.get_item, i)))
        else :
            return self.get_item(i.tolist())

    def save_to_file(self, name) :
      json.dump({'dim': self.dim, 'mapping': self.mapping}, open(name,'w'))

    @classmethod
    def load_from_file(cls, name) :
      f = json.load(open(name))
      self = cls([],0,True)
      self.dim = f['dim']
      self.mapping = f['mapping']
      return self

In [0]:
df1 = pd.read_csv('Rnn_train.csv')
df2 = pd.read_csv('Rnn_test.csv')
words = df1.Word.tolist() + df2.Word.tolist()
trans = translator(words, 10)
print(trans.dim)

In [0]:
trans.save_to_file('RNN_one_hoter.json')

In [16]:
for i,df in enumerate([df1,df2]) :
  X,Y = list(),list()
  x,y = list(),list()
  for _,row in tqdm(df.iloc[:,:3].iterrows(), total=df.shape[0]) :
    sen,word,pos = row
    if type(sen) == type('') :
      X.append(x)
      Y.append(y)
      x,y = list(),list()
    x.append(word)
    y.append(pos)
  X = X[1:]
  Y = Y[1:]
  t = 'test' if i else 'train'
  json.dump(X, open('RNN_'+t+'_x.json','w'))
  json.dump(Y, open('RNN_'+t+'_y.json','w'))